In [1]:
from keras import layers
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import winsound
from sklearn.utils import resample
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt
import os
import glob
import cv2

In [2]:
target_width, target_height = 500, 400
cv2.destroyAllWindows()

In [3]:
def read_images(path):
    image_files = glob.glob(os.path.join(path, "*.jpeg"))
    X =[]

    for image_file in image_files:
        image = cv2.imread(image_file)
        image = cv2.resize(image, (target_width, target_height))
        X.append(image)
    return X

In [4]:
# train_normal = read_images("ChestXRay2017/chest_xray/train/normal")
# train_pneumonia = read_images("ChestXRay2017/chest_xray/train/pneumonia")
# train_X = train_normal+train_pneumonia
# train_y = np.zeros(len(train_normal)).tolist() + np.ones(len(train_pneumonia)).tolist()

test_normal = read_images("ChestXRay2017/chest_xray/test/normal")
test_pneumonia = read_images("ChestXRay2017/chest_xray/test/pneumonia")
test_X = np.array(test_normal+test_pneumonia)
test_y = np.array(np.zeros(len(test_normal)).tolist() + np.ones(len(test_pneumonia)).tolist())

In [5]:
def create_model(X, y, verbose=1, epochs=100, batch_size=128):
    model = Sequential()

    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(target_height,target_width, 3))),
    model.add(layers.MaxPooling2D((2, 2))),
    model.add(layers.Dropout(0.1)),

    model.add(layers.Conv2D(32, (5, 5), activation='relu')),
    model.add(layers.MaxPooling2D((2, 2))),
    model.add(layers.Dropout(0.1)),

    model.add(layers.Flatten())

    model.add(Dense(128, activation="relu"))
    model.add(layers.Dropout(0.25))
    model.add(Dense(16, activation="relu"))
    model.add(layers.Dropout(0.15))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.00025, momentum=0.85), metrics=['accuracy'])

    history = model.fit(X, y, epochs=epochs, verbose=verbose, batch_size=batch_size, validation_split=0.2)


    return history

In [ ]:
history = create_model(test_X, test_y)

Epoch 1/100
4/4 [==============================] - 47s 12s/step - loss: 160.8603 - accuracy: 0.4469 - val_loss: 192.4041 - val_accuracy: 0.0000e+00
Epoch 2/100
1/4 [======>.......................] - ETA: 32s - loss: 140.1802 - accuracy: 0.4766

In [ ]:
loss_train = history.history['loss'][1:]
loss_val = history.history['val_loss'][1:]
plt.plot(loss_train, 'g', label='Training loss')
plt.plot(loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
plt.plot(loss_train, 'g', label='Training accuracy')
plt.plot(loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
cv2.destroyAllWindows()